In [1]:
# import libraries
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier,XGBRegressor
from sklearn.feature_selection import SelectPercentile , chi2
import warnings


# load test data 

In [2]:
df=pd.read_csv('test.csv')

In [3]:
# select features we need to get the saleprice 
features =df[[ 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF',
       'FullBath', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'Fireplaces',
       'BsmtFinSF1', 'Foundation', 'LotFrontage', 'WoodDeckSF', '2ndFlrSF',
       'OpenPorchSF']]
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OverallQual   1459 non-null   int64  
 1   GrLivArea     1459 non-null   int64  
 2   GarageCars    1458 non-null   float64
 3   TotalBsmtSF   1458 non-null   float64
 4   FullBath      1459 non-null   int64  
 5   YearBuilt     1459 non-null   int64  
 6   YearRemodAdd  1459 non-null   int64  
 7   MasVnrArea    1444 non-null   float64
 8   Fireplaces    1459 non-null   int64  
 9   BsmtFinSF1    1458 non-null   float64
 10  Foundation    1459 non-null   object 
 11  LotFrontage   1232 non-null   float64
 12  WoodDeckSF    1459 non-null   int64  
 13  2ndFlrSF      1459 non-null   int64  
 14  OpenPorchSF   1459 non-null   int64  
dtypes: float64(5), int64(9), object(1)
memory usage: 171.1+ KB


 ##### notes : 
 i found nan data 
 so i will fixed it before get saleprice 

### 1-LotFrontage

In [4]:
# data nan ( to prediction it )
df_p=df[['LotArea', 'BsmtFinSF1', 'BsmtFinSF2', 'PoolArea', 'MiscVal','LotFrontage']]
nan_lotfrontage = df.query('LotFrontage=="nan"')
df_p=df_p.drop(nan_lotfrontage.index,axis=0)


In [5]:
### sepreate  nan  to predict  it in model 
X_nan=nan_lotfrontage[['LotArea', 'BsmtFinSF1', 'BsmtFinSF2', 'PoolArea', 'MiscVal']]
X_nan=X_nan.dropna()

In [6]:
standard_scaler=StandardScaler()
X_nan2=standard_scaler.fit(X_nan)
X_nan3=X_nan2.transform(X_nan)

##  model  LotFrontage

In [7]:
# drop nan 
df_p=df_p.dropna()


In [8]:
X=df_p[['LotArea','BsmtFinSF1', 'BsmtFinSF2', 'PoolArea', 'MiscVal']]
y=df_p[['LotFrontage']]


In [9]:
standard_scaler=StandardScaler()
X=standard_scaler.fit_transform(X)
y=standard_scaler.fit_transform(y)

In [10]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.25, random_state=33)

In [11]:
# the model 
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score,f1_score,mean_squared_error

XGB=XGBRegressor(n_estimators=5000, max_depth=3, learning_rate=0.5)
XGB=XGB.fit(X,y) 
y_pred=XGB.predict(x_test)
mean_squared_error(y_test, y_pred,squared=False)


0.07568217234327627

In [12]:
value_lotfrontage=XGB.predict(X_nan3)


In [13]:
#convert  1D to 2D 
value_lotfrontage=np.reshape(value_lotfrontage,(227 ,1))

### value of nan lotfrontage

In [14]:
# get the value 
value_lotfrontage=standard_scaler.inverse_transform(value_lotfrontage)
value_lotfrontage=list(value_lotfrontage)


In [15]:
# create a new column 
X_nan.insert(loc=1, column="LotFrontage", value=value_lotfrontage)
### change the columns data type to integer  
## add the data to the real data 

X_nan['LotFrontage']=X_nan['LotFrontage'].astype('int')
X_nan
 

,LotArea,LotFrontage,BsmtFinSF1,BsmtFinSF2,PoolArea,MiscVal
6,7980,44,935.0,0.0,0,500
40,2980,40,466.0,0.0,0,0
41,2403,40,244.0,0.0,0,0
45,10456,74,506.0,0.0,0,0
47,18837,70,687.0,46.0,0,0
...,...,...,...,...,...,...
1387,11088,83,872.0,0.0,0,0
1390,21533,97,0.0,0.0,0,0
1440,50102,123,909.0,0.0,0,0
1441,8098,56,1136.0,116.0,0,0


In [16]:
list_index=X_nan.index
for i in list_index:
    df.LotFrontage[i]=X_nan.LotFrontage[i]

<ipython-input-16-be5faf3f9668>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.LotFrontage[i]=X_nan.LotFrontage[i]


### 2-MasVnrArea

In [17]:
# data nan ( to prediction it )
df_p=df[['LotArea', 'BsmtFinSF1', 'MiscVal','MasVnrArea']]
nan_MasVnrArea = df.query('MasVnrArea=="nan"')
df_p=df_p.drop(nan_MasVnrArea.index,axis=0)


In [18]:
### sepreate  nan  to predict  it in model 
X_nan=nan_MasVnrArea[['LotArea', 'BsmtFinSF1', 'MiscVal']]
X_nan=X_nan.dropna()

In [19]:
standard_scaler=StandardScaler()
X_nan2=standard_scaler.fit(X_nan)
X_nan3=X_nan2.transform(X_nan)

##  model  MasVnrArea

In [20]:
# drop nan 
df_p=df_p.dropna()


In [21]:
X=df_p[['LotArea', 'BsmtFinSF1', 'MiscVal']]
y=df_p[['MasVnrArea']]


In [22]:
standard_scaler=StandardScaler()
X=standard_scaler.fit_transform(X)
y=standard_scaler.fit_transform(y)

In [23]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.25, random_state=33)

In [24]:
# the model 
XGB=XGBRegressor(n_estimators=5000, max_depth=3, learning_rate=0.5)
XGB=XGB.fit(X,y) 
y_pred=XGB.predict(x_test)
mean_squared_error(y_test, y_pred,squared=False)

0.04561660682477453

In [25]:
# predict nan value 
value_MasVnrArea=XGB.predict(X_nan3)


In [26]:
#convert  1D to 2D 
value_MasVnrArea=np.reshape(value_MasVnrArea,(15 ,1))

### value of nan MasVnrArea


In [27]:
# get the value 
value_MasVnrArea=standard_scaler.inverse_transform(value_MasVnrArea)
value_MasVnrArea=list(value_MasVnrArea)


In [28]:
# create a new column 
X_nan.insert(loc=1, column="MasVnrArea", value=value_MasVnrArea)
### change the columns data type to integer  
## add the data to the real data 

X_nan['MasVnrArea']=X_nan['MasVnrArea'].astype('int')
X_nan.query('MasVnrArea<0')
 

,LotArea,MasVnrArea,BsmtFinSF1,MiscVal
231,12891,-126,651.0,0
422,8749,-54,0.0,0
532,7750,-31,0.0,0
544,10037,-47,666.0,0
581,7500,-33,0.0,0
908,5330,-50,0.0,0
1132,8298,-45,583.0,0
1197,12867,-9,0.0,0


In [29]:
# if MasVnrArea < 0  change it value to zero 
index_list =X_nan.query('MasVnrArea<0').index
for i in index_list:
    X_nan['MasVnrArea'][i]=0


<ipython-input-29-82b8190036b8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_nan['MasVnrArea'][i]=0


In [30]:
list_index=X_nan.index
for i in list_index:
    df.MasVnrArea[i]=X_nan.MasVnrArea[i]

<ipython-input-30-0da732494fa3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.MasVnrArea[i]=X_nan.MasVnrArea[i]


In [31]:
# select features we need to get the saleprice 
features =df[[ 'Id','OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF',
       'FullBath', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'Fireplaces',
       'BsmtFinSF1', 'Foundation', 'LotFrontage', 'WoodDeckSF', '2ndFlrSF',
       'OpenPorchSF']]
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            1459 non-null   int64  
 1   OverallQual   1459 non-null   int64  
 2   GrLivArea     1459 non-null   int64  
 3   GarageCars    1458 non-null   float64
 4   TotalBsmtSF   1458 non-null   float64
 5   FullBath      1459 non-null   int64  
 6   YearBuilt     1459 non-null   int64  
 7   YearRemodAdd  1459 non-null   int64  
 8   MasVnrArea    1459 non-null   float64
 9   Fireplaces    1459 non-null   int64  
 10  BsmtFinSF1    1458 non-null   float64
 11  Foundation    1459 non-null   object 
 12  LotFrontage   1459 non-null   float64
 13  WoodDeckSF    1459 non-null   int64  
 14  2ndFlrSF      1459 non-null   int64  
 15  OpenPorchSF   1459 non-null   int64  
dtypes: float64(5), int64(10), object(1)
memory usage: 182.5+ KB


In [32]:
#fill na  value with zero 
features.fillna(0,inplace=True)
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            1459 non-null   int64  
 1   OverallQual   1459 non-null   int64  
 2   GrLivArea     1459 non-null   int64  
 3   GarageCars    1459 non-null   float64
 4   TotalBsmtSF   1459 non-null   float64
 5   FullBath      1459 non-null   int64  
 6   YearBuilt     1459 non-null   int64  
 7   YearRemodAdd  1459 non-null   int64  
 8   MasVnrArea    1459 non-null   float64
 9   Fireplaces    1459 non-null   int64  
 10  BsmtFinSF1    1459 non-null   float64
 11  Foundation    1459 non-null   object 
 12  LotFrontage   1459 non-null   float64
 13  WoodDeckSF    1459 non-null   int64  
 14  2ndFlrSF      1459 non-null   int64  
 15  OpenPorchSF   1459 non-null   int64  
dtypes: float64(5), int64(10), object(1)
memory usage: 182.5+ KB


C:\Users\princ\anaconda3\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [33]:
# convert categorical to numerical 
enc = OrdinalEncoder()
features['Foundation']=enc.fit_transform(features[['Foundation']])

<ipython-input-33-18e1b09f033b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Foundation']=enc.fit_transform(features[['Foundation']])


In [34]:
features.to_csv('features_test.csv', index=False)